In [20]:
import tensorflow as tf
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
from tensorflow.keras.utils import load_img, img_to_array
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics import multilabel_confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score

In [14]:
!wget https://github.com/NutriMatch/MachineLearning/archive/refs/heads/master.zip

zip_ref = zipfile.ZipFile("./master.zip", 'r')
zip_ref.extractall("./")
zip_ref.close()

labels = ["ayam", "nasi", "telur", "brokoli", "ikan", "jeruk", "mie", "roti", "tahu", "tempe"]

def load_image_pixels(filename, shape):
  image = load_img(filename)
  width, height = image.size
  image = load_img(filename, target_size=shape)
  image = img_to_array(image)
  image = image.astype('float32')
  image /= 255.0
  image = np.expand_dims(image, 0)
  return image, width, height

--2023-06-14 16:26:27--  https://github.com/NutriMatch/MachineLearning/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/NutriMatch/MachineLearning/zip/refs/heads/master [following]
--2023-06-14 16:26:27--  https://codeload.github.com/NutriMatch/MachineLearning/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [    <=>             ] 214.58M  27.0MB/s    in 8.1s    

2023-06-14 16:26:36 (26.5 MB/s) - ‘master.zip’ saved [225007378]



In [16]:
!wget https://github.com/NutriMatch/MachineLearning/raw/master/NutriMatch.h5

model = tf.keras.models.load_model("NutriMatch.h5", compile=False)
folder_path = "/work/MachineLearning-master/test"

y_pred = []
y = []
for filename in os.listdir(folder_path):
    detect = []
    actual = []
    for i in labels:
        if i in filename:
            actual.append(1)
        else:
            actual.append(0)        
    input_w, input_h = 416, 416
    image, image_w, image_h = load_image_pixels(os.path.join(folder_path, filename), (input_w, input_h))
    classes = model.predict(image, batch_size=10)
    # print(filename)
    for i, n in enumerate(classes[0]):
        if n >= 0.8:
            # print(f"{labels[i]}: {n: .2f}")
            detect.append(1)
        else:
            # continue
            detect.append(0)
    
    print("")
    y.append(actual)
    y_pred.append(detect)
    
print(f"expected y:\n{y}")
print(f"predicted y:\n{y_pred}")

--2023-06-14 16:28:46--  https://github.com/NutriMatch/MachineLearning/raw/master/NutriMatch.h5
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/NutriMatch/MachineLearning/master/NutriMatch.h5 [following]
--2023-06-14 16:28:46--  https://media.githubusercontent.com/media/NutriMatch/MachineLearning/master/NutriMatch.h5
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 171532816 (164M) [application/octet-stream]
Saving to: ‘NutriMatch.h5’

NutriMatch.h5       100%[===================>] 163.59M   387MB/s    in 0.4s    

2023-06-14 16:28:48 (387 MB/s) - ‘NutriMatch.h5’ saved [171532816/

In [22]:
print(classification_report(y, y_pred, output_dict=False, target_names=labels))

              precision    recall  f1-score   support

        ayam       0.50      0.11      0.18         9
        nasi       1.00      0.26      0.42        19
       telur       0.17      0.25      0.20         4
     brokoli       1.00      0.67      0.80         3
        ikan       0.50      0.50      0.50         4
       jeruk       1.00      0.75      0.86         4
         mie       1.00      0.50      0.67         6
        roti       1.00      0.75      0.86         4
        tahu       0.20      0.20      0.20         5
       tempe       0.50      0.50      0.50         6

   micro avg       0.62      0.38      0.47        64
   macro avg       0.69      0.45      0.52        64
weighted avg       0.74      0.38      0.46        64
 samples avg       0.51      0.48      0.47        64

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in sample

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f0dcadae-0e48-4777-9b89-cc74575e8658' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>